# Experimenting simulation tools

This notebook is a playground to try some `python` tools to improve simulations.

In [1]:
import numpy as np
import numba as nb

## Numba

### Vectorize

In [2]:
grid = np.linspace(-3, 3, 5000)
x, y = np.meshgrid(grid, grid)

In [3]:
def f(x, y):
    return np.cos(x ** 2 + y ** 2) / (1 + x ** 2 + y ** 2)


%timeit np.max(f(x, y))

726 ms ± 11.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
@nb.vectorize
def f_vec(x, y):
    return np.cos(x ** 2 + y ** 2) / (1 + x ** 2 + y ** 2)


%timeit np.max(f_vec(x, y))

462 ms ± 2.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


##### Parallel

In [5]:
@nb.vectorize("float64(float64, float64)", target="parallel")
def f_vec_par(x, y):
    return np.cos(x ** 2 + y ** 2) / (1 + x ** 2 + y ** 2)


%timeit np.max(f_vec_par(x, y))

143 ms ± 12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Loops

In [6]:
def loop():
    arr = np.zeros(100)
    for j in range(len(arr)):
        for _ in range(int(1e5)):
            arr[j] += 1

    return arr


%timeit loop()

4.96 s ± 29.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
@nb.njit
def nb_loop():
    arr = np.zeros(100)
    for j in range(len(arr)):
        for _ in range(int(1e5)):
            arr[j] += 1

    return arr


%timeit nb_loop()

8.55 ms ± 92 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
@nb.njit(parallel=True)
def nb_loop_par():
    arr = np.zeros(100)
    for j in nb.prange(len(arr)):
        for _ in nb.prange(int(1e5)):
            arr[j] += 1

    return arr


%timeit nb_loop_par()

2.79 ms ± 443 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Functions as arguments

In [9]:
@nb.jit
def add1(x):
    return x + 1

@nb.jit
def bar(fn, x):
    return fn(x)

# Passing add1 into bar from interpreted code
%timeit bar(add1, 1)

11.9 µs ± 177 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [10]:
@nb.jit
def foo(x):
    return bar(add1, x)

# Passing add1 within numba compiled code.
%timeit foo(1)

164 ns ± 0.473 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
